In [7]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [4]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

This function generates output word by word - Just like in ChatGPT

In [6]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end = "", flush = True)

Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Hi [Recipient's Name],

I hope this message finds you well.

Navigating SOC 2 compliance can be a complex and time-consuming process. At ComplAI, we understand the challenges organizations face in ensuring data security and preparing for audits. Our AI-powered SaaS tool streamlines compliance management, helping you stay ahead of regulatory requirements effortlessly.

With ComplAI, you can expect:

- **Automated Reports:** Generate comprehensive compliance reports in seconds.
- **Real-Time Monitoring:** Stay updated with continuous compliance tracking.
- **Guided Workflows:** Simplify your audit preparations with step-by-step assistance.

Many organizations have already entrusted us to enhance their compliance efforts, and I would love to discuss how we can support your team in achieving a seamless experience.

Are you available for a brief call this week to explore this further?

Thank you for considering ComplAI. I look fo

In [8]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m with ComplAI, a leading provider of AI-powered tools designed to streamline SOC 2 compliance and audit preparations.

In today's regulatory landscape, maintaining compliance can be a significant challenge, often consuming valuable time and resources. Our innovative SaaS solution simplifies this process, allowing your team to effectively manage documentation, automate workflows, and stay ahead of compliance requirements.

Here's how ComplAI can benefit your organization:

1. **Automated Documentation**: Reduce manual effort and minimize errors with intelligent document generation and management.
2. **Real-Time Auditing**: Stay audit-ready with continuous monitoring and instant access to compliance data.
3. **Cost Efficiency**: Save time and resources, allowing your team to focus on core business activities.

I would love to schedule 

In [9]:
# Picks the best email out of all the generated ones 

sales_picker = Agent(
    name = "sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [11]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

    outputs = [result.final_output for result in results]
    emails = "Cold Sales Email: \n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)
    print(f"Best Sales Email: \n\n {best.final_output}")

Best Sales Email: 

 Subject: Make SOC2 Compliance Less of a Headache 😅

Hi [Recipient's Name],

I hope this email finds you somewhere between a cup of coffee and your wildest dreams of fully automated compliance. ☕️✨ 

At ComplAI, we understand that SOC2 compliance can feel like a never-ending game of “Where’s Waldo?”—just when you think you've found everything, there’s always something sneaky lurking around the corner. That’s where we come in! 

Our AI-powered platform transforms compliance from a daunting task into a walk in the park (but without the mud and the surprise dog poop). 🐕

Here are a few things we can help you with:
- **Automated Documentation:** No more digging through files like an archeologist at a dino dig.
- **Real-time Monitoring:** Think of it as your compliance fairy godmother, saying, “Bibbidi-bobbidi-boo, you're compliant—who knew?”
- **Audit Prep:** Wave goodbye to the pre-audit panic attacks. 🙌

Feel free to get back to me if you’d like to chat about how we c

Making use of Tools

In [12]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = "gpt-4o-mini"
)
sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = "gpt-4o-mini"
)
sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = "gpt-4o-mini"
)

In [13]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

Steps 2,3 - Agent and Tool interactions